# MP-100 CAPE Training on Google Colab

This notebook trains Category-Agnostic Pose Estimation (CAPE) on the MP-100 dataset using Google Colab's GPU.

## Setup Instructions
1. Enable GPU: Runtime → Change runtime type → GPU (T4 or better)
2. Run all cells in order
3. The notebook will:
   - Clone code from GitHub
   - Install dependencies
   - Authenticate to GCP
   - Mount GCS bucket with data
   - Run training with "tiny" mode


## 1. Check GPU Availability


In [ ]:
# Check GPU availability
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️  No GPU detected! Please enable GPU in Runtime > Change runtime type > GPU")


CUDA available: True
GPU: NVIDIA L4
CUDA version: 12.6
GPU Memory: 23.80 GB


In [ ]:
# Clone repository from GitHub
import os
from getpass import getpass

REPO_URL = "https://github.com/nkkrnkl/category-agnostic-pose-estimation.git"
BRANCH = "teo-branch-new-copy"
PROJECT_ROOT = "/content/category-agnostic-pose-estimation"

# Remove existing directory if it exists
if os.path.exists(PROJECT_ROOT):
    print(f"Removing existing directory: {PROJECT_ROOT}")
    !rm -rf {PROJECT_ROOT}

# For private repositories, you need to authenticate
# Option 1: Use Personal Access Token (recommended)
# Get token from: https://github.com/settings/tokens
# Create a token with 'repo' scope
print("For private repositories, you need to authenticate.")
print("Option 1: Enter your GitHub Personal Access Token")
print("  (Get one from: https://github.com/settings/tokens)")
print("Option 2: Press Enter to try without token (will fail if repo is private)")
print()

GITHUB_TOKEN = getpass("Enter GitHub Personal Access Token (or press Enter to skip): ")

if GITHUB_TOKEN.strip():
    # Use token in URL
    # Format: https://TOKEN@github.com/username/repo.git
    AUTH_REPO_URL = REPO_URL.replace("https://github.com/", f"https://{GITHUB_TOKEN}@github.com/")
    print(f"Cloning repository from {REPO_URL} (branch: {BRANCH})...")
    !git clone -b {BRANCH} {AUTH_REPO_URL} {PROJECT_ROOT}
    !git pull origin {BRANCH}
else:
    # Try without token (will work if repo is public)
    print(f"Cloning repository from {REPO_URL} (branch: {BRANCH})...")
    !git clone -b {BRANCH} {REPO_URL} {PROJECT_ROOT}

# Verify clone
if os.path.exists(PROJECT_ROOT) and os.path.exists(os.path.join(PROJECT_ROOT, ".git")):
    print(f"✅ Repository cloned successfully to {PROJECT_ROOT}")
    !cd {PROJECT_ROOT} && git branch
else:
    print("❌ Failed to clone repository")
    print("\nIf the repository is private, you need to:")
    print("1. Create a Personal Access Token at: https://github.com/settings/tokens")
    print("2. Select 'repo' scope")
    print("3. Run this cell again and paste the token when prompted")


For private repositories, you need to authenticate.
Option 1: Enter your GitHub Personal Access Token
  (Get one from: https://github.com/settings/tokens)
Option 2: Press Enter to try without token (will fail if repo is private)

Enter GitHub Personal Access Token (or press Enter to skip): ··········
Cloning repository from https://github.com/nkkrnkl/category-agnostic-pose-estimation.git (branch: teo-branch-new-copy)...
Cloning into '/content/category-agnostic-pose-estimation'...
remote: Enumerating objects: 711, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 711 (delta 30), reused 38 (delta 16), pack-reused 648 (from 1)
Receiving objects: 100% (711/711), 40.86 MiB | 14.46 MiB/s, done.
Resolving deltas: 100% (236/236), done.
Updating files: 100% (82/82), done.
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 5 (delta 4), reuse

In [ ]:
import os

PROJECT_ROOT = "/content/category-agnostic-pose-estimation"
BRANCH = "teo-branch-new-copy"

print(f"Pulling latest changes from branch {BRANCH}...")
!cd {PROJECT_ROOT} && git pull origin {BRANCH}

print("✅ Git pull complete!")

Pulling latest changes from branch teo-branch-new-copy...
From https://github.com/nkkrnkl/category-agnostic-pose-estimation
 * branch            teo-branch-new-copy -> FETCH_HEAD
Already up to date.
✅ Git pull complete!


## 3. Install Requirements


In [ ]:
# Install additional dependencies needed for plot_utils and other utilities
# (descartes, shapely, etc. - these are in requirements.txt but not requirements_cape.txt)
print("Installing additional dependencies (descartes, shapely, etc.)...")
!pip install -q descartes shapely>=1.8.0
print("✅ Additional dependencies installed!")


Installing additional dependencies (descartes, shapely, etc.)...
✅ Additional dependencies installed!


In [ ]:
# Install requirements
import os

PROJECT_ROOT = "/content/category-agnostic-pose-estimation"
REQUIREMENTS_FILE = os.path.join(PROJECT_ROOT, "requirements_cape.txt")

print("Installing requirements from requirements_cape.txt...")
!cd {PROJECT_ROOT} && pip install -q -r {REQUIREMENTS_FILE}

# Install detectron2 for CUDA 11.8 (Colab typically has CUDA 11.8)
print("\nInstalling detectron2...")
!pip install -q 'git+https://github.com/facebookresearch/detectron2.git'

print("✅ All dependencies installed!")


Installing requirements from requirements_cape.txt...

Installing detectron2...
  Preparing metadata (setup.py) ... done
✅ All dependencies installed!


## 4. Authenticate to GCP


In [ ]:
# Authenticate to GCP
from google.colab import auth

print("Authenticating to GCP...")
auth.authenticate_user()

# Set GCP project
GCP_PROJECT = "dl-category-agnostic-pose-est"
!gcloud config set project {GCP_PROJECT}

print(f"✅ Authenticated to GCP project: {GCP_PROJECT}")


Authenticating to GCP...
Are you sure you wish to set property [core/project] to 
dl-category-agnostic-pose-est?

Do you want to continue (Y/n)?  Y

Updated property [core/project].
✅ Authenticated to GCP project: dl-category-agnostic-pose-est


## 5. Mount GCS Bucket


In [ ]:
# Mount GCS bucket using gcsfuse
import os
import subprocess
import time

PROJECT_ROOT = "/content/category-agnostic-pose-estimation"
BUCKET_NAME = "dl-category-agnostic-pose-mp100-data"
MOUNT_POINT = os.path.join(PROJECT_ROOT, "Raster2Seq_internal-main", "data")

# Install gcsfuse from Google's official repository
print("Installing gcsfuse...")
# Add Google's gcsfuse repository (updated method for newer Ubuntu versions)
!export GCSFUSE_REPO=gcsfuse-`lsb_release -c -s` && \
echo "deb http://packages.cloud.google.com/apt $GCSFUSE_REPO main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list && \
curl -fsSL https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo gpg --dearmor -o /usr/share/keyrings/cloud.google.gpg && \
echo "deb [signed-by=/usr/share/keyrings/cloud.google.gpg] http://packages.cloud.google.com/apt $GCSFUSE_REPO main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list && \
sudo apt-get update && \
sudo apt-get install -y gcsfuse

# Verify installation
!which gcsfuse
print("✅ gcsfuse installed")

# Create mount point directory and parent directories
print(f"Creating mount point: {MOUNT_POINT}")
os.makedirs(os.path.dirname(MOUNT_POINT), exist_ok=True)
os.makedirs(MOUNT_POINT, exist_ok=True)

# Check if already mounted
try:
    result = subprocess.run(['mountpoint', '-q', MOUNT_POINT], capture_output=True)
    if result.returncode == 0:
        print(f"✅ Already mounted at {MOUNT_POINT}")
    else:
        # Try to unmount if exists but not properly mounted
        try:
            subprocess.run(['fusermount', '-u', MOUNT_POINT], capture_output=True, timeout=5)
        except:
            try:
                subprocess.run(['umount', MOUNT_POINT], capture_output=True, timeout=5)
            except:
                pass
except:
    pass

# Mount the bucket
print(f"Mounting gs://{BUCKET_NAME} to {MOUNT_POINT}...")
print("This may take a moment...")

# Run gcsfuse in background
# Note: In Colab, we need to run gcsfuse in background using shell &
print(f"Running: gcsfuse --implicit-dirs {BUCKET_NAME} {MOUNT_POINT}")
!nohup gcsfuse --implicit-dirs {BUCKET_NAME} {MOUNT_POINT} > /tmp/gcsfuse.log 2>&1 &

# Wait a moment for mount to initialize
print("Waiting for mount to initialize...")
time.sleep(8)  # Give it more time to mount

# Check mount status
print("\nChecking mount status...")
# Check mount log for errors
if os.path.exists("/tmp/gcsfuse.log"):
    with open("/tmp/gcsfuse.log", "r") as f:
        log_content = f.read()
        if log_content:
            print("Mount log:")
            print(log_content[-500:])  # Last 500 chars
        else:
            print("Mount log is empty (mount might still be initializing)")

# Also verify we can access the bucket directly with gsutil
print("\nVerifying bucket access with gsutil...")
!gsutil ls gs://{BUCKET_NAME}/ | head -10

# Verify mount
print(f"\nVerifying mount at: {MOUNT_POINT}")
print(f"Path exists: {os.path.exists(MOUNT_POINT)}")

# Check if actually mounted using mountpoint command
try:
    result = subprocess.run(['mountpoint', '-q', MOUNT_POINT], capture_output=True)
    is_mounted = (result.returncode == 0)
    print(f"Is mounted: {is_mounted}")
except:
    # Fallback: check mount table
    result = subprocess.run(['mount'], capture_output=True, text=True)
    is_mounted = MOUNT_POINT in result.stdout
    print(f"Is mounted (from mount table): {is_mounted}")

if os.path.exists(MOUNT_POINT) and is_mounted:
    try:
        # Try to list contents
        items = os.listdir(MOUNT_POINT)
        if len(items) > 0:
            print(f"✅ GCS bucket mounted successfully!")
            print(f"Mount point: {MOUNT_POINT}")
            print(f"Found {len(items)} items in bucket")
            # List a few items to verify
            for item in items[:10]:
                item_path = os.path.join(MOUNT_POINT, item)
                item_type = "directory" if os.path.isdir(item_path) else "file"
                print(f"   - {item} ({item_type})")
        else:
            print(f"⚠️  Mount point exists but is empty (0 items)")
            print(f"   This might indicate:")
            print(f"   1. Bucket is empty")
            print(f"   2. Mount didn't work correctly")
            print(f"   3. Permission issues")
    except PermissionError as e:
        print(f"⚠️  Permission error accessing mount: {e}")
        print("   Mount might still be initializing, wait a moment and try again")
    except Exception as e:
        print(f"⚠️  Mount point exists but cannot list contents: {e}")
        print("   This might indicate a mount issue")
        import traceback
        traceback.print_exc()
elif os.path.exists(MOUNT_POINT) and not is_mounted:
    print(f"⚠️  Directory exists but is not mounted")
    print(f"   The directory exists but gcsfuse mount is not active")
    print(f"   Trying to mount again...")
    # Try mounting again
    !nohup gcsfuse --implicit-dirs {BUCKET_NAME} {MOUNT_POINT} > /tmp/gcsfuse.log 2>&1 &
    time.sleep(5)
    # Re-check
    items = os.listdir(MOUNT_POINT) if os.path.exists(MOUNT_POINT) else []
    if len(items) > 0:
        print(f"✅ Mount successful after retry! Found {len(items)} items")
    else:
        print(f"❌ Mount still not working")
else:
    print("❌ Failed to mount GCS bucket")
    print(f"   Mount point: {MOUNT_POINT}")
    print(f"   Check:")
    print(f"   1. GCP authentication (run the GCP auth cell)")
    print(f"   2. Bucket name is correct: {BUCKET_NAME}")
    print(f"   3. You have read access to the bucket")
    print(f"   4. Check mount log: /tmp/gcsfuse.log")


Installing gcsfuse...
deb http://packages.cloud.google.com/apt gcsfuse-jammy main
gpg: cannot open '/dev/tty': No such device or address
curl: (23) Failed writing body
/usr/bin/gcsfuse
✅ gcsfuse installed
Creating mount point: /content/category-agnostic-pose-estimation/Raster2Seq_internal-main/data
Mounting gs://dl-category-agnostic-pose-mp100-data to /content/category-agnostic-pose-estimation/Raster2Seq_internal-main/data...
This may take a moment...
Running: gcsfuse --implicit-dirs dl-category-agnostic-pose-mp100-data /content/category-agnostic-pose-estimation/Raster2Seq_internal-main/data
Waiting for mount to initialize...

Checking mount status...
Mount log:
ckSizeMb":16,"EnableBufferedRead":false,"GlobalMaxBlocks":40,"InactiveStreamTimeout":10000000000,"MaxBlocksPerHandle":20,"MinBlocksPerHandle":4,"RandomSeekThreshold":3,"StartBlocksPerHandle":1},"Write":{"BlockSizeMb":32,"CreateEmptyFile":false,"EnableRapidAppends":true,"EnableStreamingWrites":true,"FinalizeFileForRapid":false,"

## 6. Create Data Symlink


In [ ]:
# Create symlink from data to mounted GCS bucket (as expected by START_TRAINING.sh)
import os

PROJECT_ROOT = "/content/category-agnostic-pose-estimation"
MOUNTED_DATA = os.path.join(PROJECT_ROOT, "Raster2Seq_internal-main", "data")
DATA_SYMLINK = os.path.join(PROJECT_ROOT, "data")

print(f"Checking mount point: {MOUNTED_DATA}")
print(f"  Exists: {os.path.exists(MOUNTED_DATA)}")
if os.path.exists(MOUNTED_DATA):
    print(f"  Is directory: {os.path.isdir(MOUNTED_DATA)}")
    try:
        items = os.listdir(MOUNTED_DATA)
        print(f"  Can list contents: Yes ({len(items)} items)")
    except Exception as e:
        print(f"  Can list contents: No ({e})")

# Remove existing symlink or directory if it exists
if os.path.exists(DATA_SYMLINK):
    if os.path.islink(DATA_SYMLINK):
        print(f"Removing existing symlink: {DATA_SYMLINK}")
        os.unlink(DATA_SYMLINK)
    elif os.path.isdir(DATA_SYMLINK):
        print(f"Warning: {DATA_SYMLINK} exists as a directory (not a symlink)")
        print("   Removing it to create symlink...")
        import shutil
        shutil.rmtree(DATA_SYMLINK)
    else:
        print(f"Warning: {DATA_SYMLINK} exists and is not a symlink or directory")
        os.remove(DATA_SYMLINK)

# Create symlink
if os.path.exists(MOUNTED_DATA) and os.path.isdir(MOUNTED_DATA):
    try:
        # Use absolute path for symlink target
        MOUNTED_DATA_ABS = os.path.abspath(MOUNTED_DATA)
        print(f"\nCreating symlink:")
        print(f"  From: {DATA_SYMLINK}")
        print(f"  To: {MOUNTED_DATA_ABS}")
        os.symlink(MOUNTED_DATA_ABS, DATA_SYMLINK)
        print(f"✅ Created symlink: {DATA_SYMLINK} -> {MOUNTED_DATA_ABS}")

        # Verify symlink
        if os.path.exists(DATA_SYMLINK):
            print(f"✅ Symlink verified: {DATA_SYMLINK}")
            print(f"  Is symlink: {os.path.islink(DATA_SYMLINK)}")
            # Try to list contents through symlink
            try:
                items = os.listdir(DATA_SYMLINK)
                print(f"✅ Can access {len(items)} items through symlink")
                print(f"   First 5 items: {items[:5]}")
            except Exception as e:
                print(f"⚠️  Symlink exists but cannot access contents: {e}")
        else:
            print(f"❌ Symlink creation failed - path does not exist after creation")
    except Exception as e:
        print(f"❌ Error creating symlink: {e}")
        print(f"   Source: {MOUNTED_DATA}")
        print(f"   Target: {DATA_SYMLINK}")
        import traceback
        traceback.print_exc()
else:
    print(f"❌ Mounted data not found at {MOUNTED_DATA}")
    print(f"   Please check that GCS bucket is mounted correctly")
    print(f"   Run the mount cell above and check for errors")
    print(f"   Mount point should exist and be accessible")


Checking mount point: /content/category-agnostic-pose-estimation/Raster2Seq_internal-main/data
  Exists: True
  Is directory: True
  Can list contents: Yes (100 items)

Creating symlink:
  From: /content/category-agnostic-pose-estimation/data
  To: /content/category-agnostic-pose-estimation/Raster2Seq_internal-main/data
✅ Created symlink: /content/category-agnostic-pose-estimation/data -> /content/category-agnostic-pose-estimation/Raster2Seq_internal-main/data
✅ Symlink verified: /content/category-agnostic-pose-estimation/data
  Is symlink: True
✅ Can access 100 items through symlink
   First 5 items: ['alpaca_face', 'amur_tiger_body', 'annotations', 'antelope_body', 'arcticwolf_face']


In [ ]:
## 7. Run Single Image Training

This section trains the model on a **single image** from a specific category for 20 epochs.
This is useful for:
- Quick overfitting test to verify the model can learn
- Debugging the training pipeline
- Testing on Colab GPU

**Training Configuration:**
- Single image mode: Enabled
- Category: 40 (zebra) - you can change this
- Epochs: 20
- All logs will be saved to `training_logs.txt`


# Configure single image training parameters


In [ ]:
# Configure single image training
import os
from datetime import datetime

PROJECT_ROOT = "/content/category-agnostic-pose-estimation"

# Training configuration
SINGLE_IMAGE_CATEGORY = 40  # Category ID (40 = zebra, change if needed)
EPOCHS = 20
BATCH_SIZE = 1
NUM_QUERIES_PER_EPISODE = 1
EPISODES_PER_EPOCH = 20

# Output directories
OUTPUT_DIR = os.path.join(PROJECT_ROOT, "output", "single_image_colab")
LOG_FILE = os.path.join(OUTPUT_DIR, "training_logs.txt")

# Create output directory
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("=" * 80)
print("Single Image Training Configuration")
print("=" * 80)
print(f"Category ID:        {SINGLE_IMAGE_CATEGORY}")
print(f"Epochs:            {EPOCHS}")
print(f"Batch size:        {BATCH_SIZE}")
print(f"Queries/episode:   {NUM_QUERIES_PER_EPISODE}")
print(f"Episodes/epoch:    {EPISODES_PER_EPOCH}")
print(f"Output directory:  {OUTPUT_DIR}")
print(f"Log file:          {LOG_FILE}")
print("=" * 80)
print()


Starting training with 'tiny' mode...
This will run 5 epochs with batch_size 8 (~30-60 min)
  Category-Agnostic Pose Estimation (CAPE) Training

Training setup:
  - 56 training categories (episodic sampling)
  - 14 test categories (unseen, for evaluation)
  - Support pose graph conditioning
  - Raster2Seq framework

🚀 NORMAL MODE: Full training configuration

Configuration:
  Epochs:              300
  Episodes/epoch:      1000
  Batch size:          2
  Queries/episode:     2
  Output directory:    output/cape_episodic_20251123_232500

✓ All required files found

Python version:
Python 3.12.12

✓ PyTorch found

Detected device: cuda:0

✓ CUDA detected - Using GPU for training
  GPU: NVIDIA L4

  Starting CAPE Training...

Category-Agnostic Pose Estimation (CAPE) - Episodic Training

Mode: Episodic meta-learning with support pose graphs
Support encoder layers: 3
Fusion method: cross_attention
Queries per episode: 2
Episodes per epoch: 1000

Using device: cuda:0
  GPU: NVIDIA L4
  CUDA 

# Run Training with Logging


In [ ]:
# Run training on single image with full logging
import subprocess
import sys
from datetime import datetime

PROJECT_ROOT = "/content/category-agnostic-pose-estimation"
OUTPUT_DIR = os.path.join(PROJECT_ROOT, "output", "single_image_colab")
LOG_FILE = os.path.join(OUTPUT_DIR, "training_logs.txt")

# Build training command
cmd = [
    sys.executable, "-m", "models.train_cape_episodic",
    "--debug_single_image", str(SINGLE_IMAGE_CATEGORY),
    "--dataset_root", PROJECT_ROOT,
    "--category_split_file", os.path.join(PROJECT_ROOT, "category_splits.json"),
    "--output_dir", OUTPUT_DIR,
    "--device", "cuda:0",
    "--epochs", str(EPOCHS),
    "--batch_size", str(BATCH_SIZE),
    "--num_queries_per_episode", str(NUM_QUERIES_PER_EPISODE),
    "--episodes_per_epoch", str(EPISODES_PER_EPOCH),
    "--lr", "1e-4",
    "--lr_backbone", "1e-5",
    "--weight_decay", "1e-4",
    "--clip_max_norm", "0.1",
    "--support_encoder_layers", "3",
    "--support_fusion_method", "cross_attention",
    "--backbone", "resnet50",
    "--hidden_dim", "256",
    "--nheads", "8",
    "--enc_layers", "6",
    "--dec_layers", "6",
    "--dim_feedforward", "1024",
    "--dropout", "0.1",
    "--image_size", "256",
    "--vocab_size", "2000",
    "--seq_len", "200",
    "--num_queries", "200",
    "--num_polys", "1",
    "--cls_loss_coef", "2.0",
    "--coords_loss_coef", "5.0",
    "--room_cls_loss_coef", "0.0",
    "--semantic_classes", "70",
    "--num_feature_levels", "4",
    "--dec_n_points", "4",
    "--enc_n_points", "4",
    "--aux_loss",
    "--with_poly_refine",
    "--num_workers", "2",
    "--seed", "42",
    "--print_freq", "5",
    "--use_amp",  # Enable mixed precision for faster training
    "--cudnn_benchmark",  # Enable cuDNN benchmark
    "--job_name", f"single_image_colab_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
]

print("Starting training...")
print(f"Command: {' '.join(cmd)}")
print(f"Logging to: {LOG_FILE}")
print("=" * 80)
print()

# Change to project directory
os.chdir(PROJECT_ROOT)

# Run training with logging to both stdout and file
with open(LOG_FILE, 'w') as log_file:
    # Write header to log file
    log_file.write("=" * 80 + "\n")
    log_file.write(f"Single Image Training Log\n")
    log_file.write(f"Started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    log_file.write(f"Category: {SINGLE_IMAGE_CATEGORY}\n")
    log_file.write(f"Epochs: {EPOCHS}\n")
    log_file.write("=" * 80 + "\n\n")
    log_file.flush()
    
    # Run process and stream output to both stdout and file
    process = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        universal_newlines=True,
        bufsize=1
    )
    
    # Stream output in real-time
    for line in process.stdout:
        print(line, end='')  # Print to notebook
        log_file.write(line)  # Write to log file
        log_file.flush()  # Ensure immediate write
    
    # Wait for process to complete
    return_code = process.wait()
    
    # Write footer to log file
    log_file.write("\n" + "=" * 80 + "\n")
    log_file.write(f"Training completed: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    log_file.write(f"Return code: {return_code}\n")
    log_file.write("=" * 80 + "\n")

if return_code == 0:
    print("\n" + "=" * 80)
    print("✅ Training completed successfully!")
    print(f"Checkpoints saved to: {OUTPUT_DIR}")
    print(f"Full logs saved to: {LOG_FILE}")
    print("=" * 80)
else:
    print("\n" + "=" * 80)
    print(f"❌ Training failed with return code: {return_code}")
    print(f"Check logs at: {LOG_FILE}")
    print("=" * 80)


In [ ]:
## 8. Check Training Results


In [ ]:
# Check training results and find best checkpoint
import os
import glob

PROJECT_ROOT = "/content/category-agnostic-pose-estimation"
OUTPUT_DIR = os.path.join(PROJECT_ROOT, "output", "single_image_colab")

print("Checking training results...")
print(f"Output directory: {OUTPUT_DIR}")
print()

# List all checkpoints
checkpoints = glob.glob(os.path.join(OUTPUT_DIR, "*.pth"))
if checkpoints:
    print(f"Found {len(checkpoints)} checkpoint(s):")
    for ckpt in sorted(checkpoints):
        size_mb = os.path.getsize(ckpt) / (1024 * 1024)
        print(f"  - {os.path.basename(ckpt)} ({size_mb:.1f} MB)")
    
    # Find best checkpoint
    best_ckpts = glob.glob(os.path.join(OUTPUT_DIR, "checkpoint_best_pck*.pth"))
    if best_ckpts:
        best_ckpt = sorted(best_ckpts)[-1]  # Get most recent
        print(f"\n✅ Best checkpoint: {os.path.basename(best_ckpt)}")
        BEST_CHECKPOINT = best_ckpt
    else:
        # Use most recent checkpoint
        best_ckpt = sorted(checkpoints, key=os.path.getmtime)[-1]
        print(f"\n⚠️  No 'best' checkpoint found, using most recent: {os.path.basename(best_ckpt)}")
        BEST_CHECKPOINT = best_ckpt
else:
    print("❌ No checkpoints found!")
    BEST_CHECKPOINT = None

# Show log file info
LOG_FILE = os.path.join(OUTPUT_DIR, "training_logs.txt")
if os.path.exists(LOG_FILE):
    size_mb = os.path.getsize(LOG_FILE) / (1024 * 1024)
    print(f"\n📄 Training log: {os.path.basename(LOG_FILE)} ({size_mb:.2f} MB)")
    
    # Show last few lines of log
    print("\nLast 20 lines of training log:")
    print("-" * 80)
    with open(LOG_FILE, 'r') as f:
        lines = f.readlines()
        for line in lines[-20:]:
            print(line.rstrip())
else:
    print("\n⚠️  Training log not found")

print("\n" + "=" * 80)


## 9. Visualize Predictions

This section visualizes the model's predictions on the single training image.
We'll use the trained checkpoint to generate predictions and visualize them.

In [ ]:
# Visualize predictions using the trained model
import os
import sys
import subprocess
from datetime import datetime

PROJECT_ROOT = "/content/category-agnostic-pose-estimation"
OUTPUT_DIR = os.path.join(PROJECT_ROOT, "output", "single_image_colab")
VISUALIZATION_DIR = os.path.join(OUTPUT_DIR, "visualizations")

# Find best checkpoint
import glob
best_ckpts = glob.glob(os.path.join(OUTPUT_DIR, "checkpoint_best_pck*.pth"))
if not best_ckpts:
    # Try any checkpoint
    all_ckpts = glob.glob(os.path.join(OUTPUT_DIR, "*.pth"))
    if all_ckpts:
        best_ckpts = [sorted(all_ckpts, key=os.path.getmtime)[-1]]

if not best_ckpts:
    print("❌ No checkpoint found! Please run training first.")
    sys.exit(1)

CHECKPOINT = sorted(best_ckpts)[-1]
print("=" * 80)
print("CAPE Prediction Visualization")
print("=" * 80)
print(f"Checkpoint: {os.path.basename(CHECKPOINT)}")
print(f"Output:     {VISUALIZATION_DIR}")
print("=" * 80)
print()

# Create visualization directory
os.makedirs(VISUALIZATION_DIR, exist_ok=True)

# Build visualization command
cmd = [
    sys.executable, "-m", "models.visualize_cape_predictions",
    "--checkpoint", CHECKPOINT,
    "--dataset_root", PROJECT_ROOT,
    "--device", "cuda:0",
    "--num_samples", "1",  # Just visualize the single training image
    "--output_dir", VISUALIZATION_DIR
]

print("Running visualization...")
print(f"Command: {' '.join(cmd)}")
print()

# Change to project directory
os.chdir(PROJECT_ROOT)

# Run visualization
try:
    result = subprocess.run(cmd, check=True, capture_output=True, text=True)
    print(result.stdout)
    if result.stderr:
        print("Warnings/Errors:")
        print(result.stderr)
    
    print("\n" + "=" * 80)
    print("✅ Visualization complete!")
    print(f"Results saved to: {VISUALIZATION_DIR}")
    print("=" * 80)
    
    # List generated visualizations
    vis_files = glob.glob(os.path.join(VISUALIZATION_DIR, "*.png"))
    if vis_files:
        print(f"\nGenerated {len(vis_files)} visualization(s):")
        for vis_file in sorted(vis_files):
            print(f"  - {os.path.basename(vis_file)}")
    
except subprocess.CalledProcessError as e:
    print(f"\n❌ Visualization failed with return code: {e.returncode}")
    print("STDOUT:")
    print(e.stdout)
    print("\nSTDERR:")
    print(e.stderr)
except Exception as e:
    print(f"\n❌ Error during visualization: {e}")
    import traceback
    traceback.print_exc()


In [ ]:
# Download results from Colab
from google.colab import files
import os
import shutil
import zipfile
import glob

PROJECT_ROOT = "/content/category-agnostic-pose-estimation"
OUTPUT_DIR = os.path.join(PROJECT_ROOT, "output", "single_image_colab")

print("Preparing results for download...")
print(f"Output directory: {OUTPUT_DIR}")
print()

# Create a zip file with all results
ZIP_FILE = "/content/single_image_training_results.zip"

with zipfile.ZipFile(ZIP_FILE, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # Add training logs
    log_file = os.path.join(OUTPUT_DIR, "training_logs.txt")
    if os.path.exists(log_file):
        zipf.write(log_file, "training_logs.txt")
        print(f"✓ Added training logs")
    
    # Add checkpoints (only best one to save space)
    best_ckpts = glob.glob(os.path.join(OUTPUT_DIR, "checkpoint_best_pck*.pth"))
    if best_ckpts:
        best_ckpt = sorted(best_ckpts)[-1]
        zipf.write(best_ckpt, f"checkpoints/{os.path.basename(best_ckpt)}")
        print(f"✓ Added best checkpoint: {os.path.basename(best_ckpt)}")
    
    # Add visualizations
    vis_dir = os.path.join(OUTPUT_DIR, "visualizations")
    if os.path.exists(vis_dir):
        for root, dirs, files in os.walk(vis_dir):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, OUTPUT_DIR)
                zipf.write(file_path, arcname)
        print(f"✓ Added visualizations")

print(f"\n✅ Created zip file: {ZIP_FILE}")
print(f"Size: {os.path.getsize(ZIP_FILE) / (1024*1024):.2f} MB")
print("\nDownloading...")
files.download(ZIP_FILE)

print("\n✅ Download complete!")
print("\nThe zip file contains:")
print("  - training_logs.txt (full training output)")
print("  - checkpoints/ (best model checkpoint)")
print("  - visualizations/ (prediction visualizations)")
